In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import urllib.parse

# — CONFIG —  
TEXTS_CSV     = "post_match_texts.csv"  
RESPONSES_CSV = "app_responses.csv"

# — LOAD DATA —  
texts_df = pd.read_csv(TEXTS_CSV, encoding='cp1252', engine='python')
try:
    resp_df = pd.read_csv(RESPONSES_CSV)
except FileNotFoundError:
    resp_df = pd.DataFrame(columns=['text_id', 'sentiment'])

def get_unseen():
    seen = set(resp_df['text_id'])
    return texts_df[~texts_df['text_id'].isin(seen)]

# — WIDGETS —  
out       = widgets.HTML()  
sentiment = widgets.RadioButtons(options=['Positive','Neutral','Negative'],
                                 description='Your rating:')  
btn_submit   = widgets.Button(description='Submit & Next') 

def make_csv_data_uri(df, filename="app_responses.csv"):
    # Turn the DataFrame into CSV text
    csv_text = df.to_csv(index=False)
    # URL-encode it
    uri = urllib.parse.quote(csv_text)
    # Build the data URI
    return f"data:text/csv;charset=utf-8,{uri}"

# — DOWNLOAD LINK WIDGET —  
download_link = widgets.HTML()
display(download_link)

def update_download_link():
    # Only if the in-memory DataFrame has rows
    if not resp_df.empty:
        data_uri = make_csv_data_uri(resp_df)
        download_link.value = f'''
        <div style="margin-top:1em">
          <a href="{data_uri}" download="app_responses.csv"
             style="padding:8px 12px;
                    background:#28a745;
                    color:white;
                    text-decoration:none;
                    border-radius:4px;">
            📥 Download all responses
          </a>
        </div>
        '''
    else:
        download_link.value = ''

# Call once at startup and again after each submit
update_download_link()

current_id = None

def load_next(_=None):
    global current_id
    unseen = get_unseen()
    if unseen.empty:
        out.value = "<h3>✅ You’ve rated every text!</h3>"
        btn_submit.disabled = True
        return
    row = unseen.sample(1).iloc[0]
    current_id = row['text_id']
    out.value = f"<b>Text #{current_id}</b><p>{row['processed_tex']}</p>"
    sentiment.value = None

def on_submit(_):
    global resp_df
    if sentiment.value is None:
        return
    # record response
    resp_df.loc[len(resp_df)] = {
        'text_id': current_id,
        'sentiment': sentiment.value
    }
    resp_df.to_csv(RESPONSES_CSV, index=False)
    update_download_link()   # <-- refreshes the HTML widget in place
    load_next()

btn_submit.on_click(on_submit)

# — LAYOUT & START —  
display(out, sentiment, btn_submit)  
load_next()